ПОДКЛЮЧЕНИЕ НЕОБХОДИМОЙ БИБЛИОТЕКИ



In [30]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.208 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
Setup complete ✅ (8 CPUs, 51.0 GB RAM, 31.3/166.8 GB disk)


ПОДКЛЮЧЕНИЕ К ХРАНИЛИЩУ ДАННЫХ С ВИДЕО


In [31]:
#
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


ЗАГРУЗКА ДАННЫХ С САЙТА ROBOFLOW

In [32]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="13nMH8aJ47Vi5S6NszrR")
project = rf.workspace("inna-bobylina-osm6z").project("metwel")
dataset = project.version(7).download("yolov8")


loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.0.208, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to MetWel-7 in yolov8:: 100%|██████████| 2932/2932 [00:00<00:00, 4347.79it/s]


TRAIN MODEL

In [23]:
!yolo task=detect model=/content/runs/detect/train9/weights/best.pt mode=train data=/content/MetWel-7/data.yaml epochs=100 imgsz=640 batch=32 optimize=True

Ultralytics YOLOv8.0.208 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
engine/trainer: task=detect, mode=train, model=/content/runs/detect/train9/weights/best.pt, data=/content/MetWel-7/data.yaml, epochs=100, patience=50, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train11, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, 

PREDICT (ПРЕДСКАЗАНИЕ) НА ВИДОСЕ ДЛИТЕЛЬНОСТЬЮ 5 МИНУТ

In [ ]:
!yolo predict model=yolov8m.pt source='/content/drive/MyDrive/Data hackaton Ставрополь/Data/video 9 hours/5 минут/1-1.mp4'

Выходные данные были обрезаны до нескольких последних строк (5000).
video 1/1 (4004/9000) /content/drive/MyDrive/Data hackaton Ставрополь/Data/video 9 hours/5 минут/1-1.mp4: 384x640 3 persons, 1 truck, 1 boat, 6.7ms
video 1/1 (4005/9000) /content/drive/MyDrive/Data hackaton Ставрополь/Data/video 9 hours/5 минут/1-1.mp4: 384x640 4 persons, 1 truck, 1 boat, 7.9ms
video 1/1 (4006/9000) /content/drive/MyDrive/Data hackaton Ставрополь/Data/video 9 hours/5 минут/1-1.mp4: 384x640 2 persons, 1 truck, 1 boat, 6.3ms
video 1/1 (4007/9000) /content/drive/MyDrive/Data hackaton Ставрополь/Data/video 9 hours/5 минут/1-1.mp4: 384x640 3 persons, 1 truck, 1 boat, 6.4ms
video 1/1 (4008/9000) /content/drive/MyDrive/Data hackaton Ставрополь/Data/video 9 hours/5 минут/1-1.mp4: 384x640 4 persons, 2 boats, 6.5ms
video 1/1 (4009/9000) /content/drive/MyDrive/Data hackaton Ставрополь/Data/video 9 hours/5 минут/1-1.mp4: 384x640 5 persons, 2 boats, 6.4ms
video 1/1 (4010/9000) /content/drive/MyDrive/Data hackaton С

МОДУЛЬ ПОДСЧЕТА ВРЕМЕНИ ДЕТЕКЦИИ

In [33]:
import time
import pandas as pd
from ultralytics import YOLO

model = YOLO("/content/runs/detect/train9/weights/best.pt")
class_labels = ['action', 'person', 'welding']
class_index_map = {label: i for i, label in enumerate(class_labels)}
start_time = time.time()
objects = model('/content/drive/MyDrive/Data/IMG_5946.MOV', save=True, show=True, conf=0.7, save_txt=True)
end_time = time.time()
class_durations = {}

for obj in objects.xyxy:
    for pred in obj:
        class_label = int(pred[5])
        confidence = pred[4] if len(pred) > 4 else 0
        label = class_labels[class_label]
        if label not in class_durations:
            class_durations[label] = 0
        class_durations[label] += (end_time - start_time)

if 'action' not in class_durations and 'welding' not in class_durations:
    class_durations['simple'] = end_time - start_time

for class_label, duration in class_durations.items():
    print(f"Detected {class_label}: {duration} seconds")

df = pd.DataFrame(columns=['Class', 'Duration'])

for class_label, duration in class_durations.items():
    df = df.append({'Class': class_label, 'Duration': duration}, ignore_index=True)

df.to_excel('class_durations.xlsx', index=False)

WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (1/2985) /content/drive/MyDrive/Data/IMG_5946.MOV: 384x640 1 people, 15.1ms
video 1/1 (2/2985) /content/drive/MyDrive/Data/IMG_5946.MOV: 384x640 1 people, 9.4ms
video 1/1 (3/2985) /content/drive/MyDrive/Data/IMG_5946.MOV: 384x640 1 people, 11.1ms
video 1/1 (4/2985) /content/drive/MyDrive/Data/IMG_5946.MOV: 384x640 1 people, 11.4ms
video 1/1 (5/2985) /content/drive/MyDri

AttributeError: ignored

ОБЕРТКА КОДА


In [ ]:
!pip install ultralytics
from ultralytics import YOLO

model_name = 'yolov8m' #@param ["yolov8n", "yolov8s", "yolov8m", "yolov8l", "yolov8x"]
input_width = 640 #@param {type:"slider", min:32, max:4096, step:32}
input_height = 480 #@param {type:"slider", min:32, max:4096, step:32}
optimize_cpu = False

model = YOLO(f"{model_name}.pt")
model.export(format="onnx", imgsz=[input_height,input_width], optimize=optimize_cpu)

In [ ]:
%cd /content
from google.colab import files

files.download(f'{model_name}.onnx')

In [34]:
!git init

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/
